In [5]:
//creation d un dataframe
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`sh.almond::almond-spark:0.10.9`
import $ivy.`org.apache.spark::spark-mllib:2.4.5`

import $ivy.$                                  

import $ivy.$                               

import $ivy.$                                    

In [6]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
import spark.implicits._


Getting spark JARs
Creating SparkSession


Spark UI

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.functions._

spark: SparkSession = org.apache.spark.sql.SparkSession@6c75533e
import spark.implicits._


# Importation du DataFrame

In [33]:
//import org.apache.spark.implicits._

val path = "data/archive/*"
val df = spark
        .read
        .option("header","true")
        .csv(path)

csv at cmd32.sc:5 
 (kill)

0 / 1

csv at cmd32.sc:5 
 (kill)

0 / 1

path: String = "data/archive/*"
df: DataFrame = [Order ID: string, Product: string ... 4 more fields]

In [8]:
df.count()
df.columns
df.printSchema()

count at cmd7.sc:1 
 (kill)

0 / 12

count at cmd7.sc:1 
 (kill)

0 / 12

count at cmd7.sc:1 
 (kill)

0 / 1

count at cmd7.sc:1 
 (kill)

0 / 1

root
 |-- Order ID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: string (nullable = true)
 |-- Price Each: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)



res7_0: Long = 186850L
res7_1: Array[String] = Array(
  "Order ID",
  "Product",
  "Quantity Ordered",
  "Price Each",
  "Order Date",
  "Purchase Address"
)

## Encodage de la colone "Order Date" en timestamp

In [9]:
import org.apache.spark.sql.functions.to_timestamp

val dfWithTimestamp = df.withColumn("Order Date", to_timestamp(col("Order Date"), "MM/dd/yyyy HH:mm"))


import org.apache.spark.sql.functions.to_timestamp


dfWithTimestamp: DataFrame = [Order ID: string, Product: string ... 4 more fields]

In [25]:
import org.apache.spark.sql.functions.{to_timestamp, year}

val dfWithTimestamp = df.withColumn("Order Date", to_timestamp(col("Order Date"), "MM/dd/yyyy HH:mm"))


import org.apache.spark.sql.functions.{to_timestamp, year}


dfWithTimestamp: DataFrame = [Order ID: string, Product: string ... 4 more fields]

### Q1: Quelle année a été la meilleure en termes de ventes ? Combien a-t-on gagné cette année-là ?

In [11]:
// Chiffre d'affaire par mois trier par ordre descroissant
val dfAnnee = dfWithTimestamp
  .groupBy(year(col("Order Date")).as("Annee"))
  .agg(sum(col("Quantity Ordered") * col("Price Each")).as("Total"))
  .orderBy(desc("Total"))

  

// On prend le premier element de la colone
val bestAnnee = dfAnnee.first().getAs[Int]("Annee")
val total = dfAnnee.first().getAs[Double]("Total")

println(s"La meilleur annee en terme de vente est 20$bestAnnee avec un total de $total dollars.")

first at cmd10.sc:9 
 (kill)

0 / 12

first at cmd10.sc:9 
 (kill)

0 / 12

first at cmd10.sc:9 
 (kill)

...

0 / 200

first at cmd10.sc:10 
 (kill)

0 / 200

La meilleur annee en terme de vente est 2019 avec un total de 3.379903641000492E7 dollars.


dfAnnee: Dataset[Row] = [Annee: int, Total: double]
bestAnnee: Int = 19
total: Double = 3.379903641000492E7

### Q2: Quel mois a été le meilleur en termes de ventes ? Combien a-t-on gagné ce mois-là ?

In [12]:
// Chiffre d'affaire par mois trier par ordre descroissant
val dfMois = dfWithTimestamp
  .groupBy(month(col("Order Date")).alias("Mois"))
  .agg(sum(col("Quantity Ordered") * col("Price Each")).as("Total"))
  .orderBy(desc("Total"))
  
// On prend le premier element de la colone
val bestMois = dfMois.first().getAs[Int]("Mois")
val total = dfMois.first().getAs[Double]("Total")

println(s"Le mois le plus rentable est le mois numéro $bestMois avec un chiffre d'affaires total de $total dollars.")


first at cmd11.sc:7 
 (kill)

0 / 12

first at cmd11.sc:7 
 (kill)

0 / 12

first at cmd11.sc:7 
 (kill)

...

0 / 200

first at cmd11.sc:8 
 (kill)

0 / 200

Le mois le plus rentable est le mois numéro 11 avec un chiffre d'affaires total de 3715537.459999819 dollars.


dfMois: Dataset[Row] = [Mois: int, Total: double]
bestMois: Int = 11
total: Double = 3715537.459999819

### Q3: Quelle ville a enregistré le plus grand nombre de ventes ?

In [13]:

// On split avec comme separateur "," et on prend le deuxieme element du tableau pour avoir la ville 
val dfCity = df.withColumn("City", split(col("Purchase Address"), ",")(1))

// Nombre de ventes par ville trier par ordre descroissant
val dfVenteByCity = dfCity
  .groupBy("City")
  .agg(count("City").alias("Total"))
  .orderBy(desc("Total"))

// On prend le premier element de la colone
val bestCity = dfVenteByCity.first().getAs[String]("City")
val total = dfVenteByCity.first().getAs[Long]("Total")

println(s"La ville qui a enregistré le plus grand nombre de ventes est $bestCity avec un total de $total ventes.")


first at cmd12.sc:10 
 (kill)

0 / 12

first at cmd12.sc:10 
 (kill)

0 / 12

first at cmd12.sc:10 
 (kill)

...

0 / 200

first at cmd12.sc:11 
 (kill)

0 / 200

La ville qui a enregistré le plus grand nombre de ventes est  San Francisco avec un total de 44732 ventes.


dfCity: DataFrame = [Order ID: string, Product: string ... 5 more fields]
dfVenteByCity: Dataset[Row] = [City: string, Total: bigint]
bestCity: String = " San Francisco"
total: Long = 44732L

### Q4:À quelle heure devraient-ils diffuser des publicités pour maximiser les chances que lesclients achètent le produit ?


In [15]:

val hourDF = dfWithTimestamp.select(hour($"Order Date").alias("Hour"))

val bestHour = hourDF.groupBy("Hour").count().orderBy(desc("count")).first().getAs[Int]("Hour")

println(s"L'heure optimale pour diffuser des publicités est $bestHour heure.")


first at cmd14.sc:3 
 (kill)

0 / 12

first at cmd14.sc:3 
 (kill)

0 / 12

first at cmd14.sc:3 
 (kill)

0 / 200

first at cmd14.sc:3 
 (kill)

0 / 200

L'heure optimale pour diffuser des publicités est 19 heure.


hourDF: DataFrame = [Hour: int]
bestHour: Int = 19

### Q5:Quels produits sont le plus souvent vendus ensemble ?

In [40]:


val df1 = dfWithTimestamp.groupBy("Order ID")
  .agg(collect_set("Product").as("Products"))


//les paires de produits pour chaque commande
val Pairs = df1.flatMap { row =>
  val products = row.getAs[Seq[String]]("Products")
  products.combinations(2).map(pair => (pair(0), pair(1)))
}.toDF("Prod 1", "Prod 2")


// Compter les occurrences des paires de produits
val PairCounts = Pairs.groupBy("Prod 1", "Prod 2")
  .count()
  .withColumn("Occurrences", col("count").cast("Int"))
  .drop("count")


// Trier
val resultat = PairCounts.orderBy(col("Occurrences").desc)

// Affichez les 10 premiers
resultat.show(truncate=false)

0 / 12

show at cmd39.sc:23 
 (kill)

0 / 12

show at cmd39.sc:23 
 (kill)

0 / 200

...

0 / 200

show at cmd39.sc:23 
 (kill)

0 / 200

+------------------------+--------------------------+-----------+
|Prod 1                  |Prod 2                    |Occurrences|
+------------------------+--------------------------+-----------+
|Lightning Charging Cable|iPhone                    |1011       |
|USB-C Charging Cable    |Google Phone              |997        |
|iPhone                  |Wired Headphones          |462        |
|Wired Headphones        |Google Phone              |422        |
|Apple Airpods Headphones|iPhone                    |373        |
|Vareebadd Phone         |USB-C Charging Cable      |368        |
|Google Phone            |Bose SoundSport Headphones|228        |
|USB-C Charging Cable    |Wired Headphones          |203        |
|Vareebadd Phone         |Wired Headphones          |149        |
|Lightning Charging Cable|Wired Headphones          |129        |
|Lightning Charging Cable|AA Batteries (4-pack)     |106        |
|USB-C Charging Cable    |Bose SoundSport Headphones|102        |
|Apple Air

df1: DataFrame = [Order ID: string, Products: array<string>]
Pairs: DataFrame = [Prod 1: string, Prod 2: string]
PairCounts: DataFrame = [Prod 1: string, Prod 2: string ... 1 more field]
resultat: Dataset[Row] = [Prod 1: string, Prod 2: string ... 1 more field]

## Q6:Quel produit s'est le plus vendu?

In [37]:
// Calcul du total des quantités vendues par produit
val Vente = dfWithTimestamp.groupBy("Product")
  .agg(sum("Quantity Ordered").as("Total"))
  .orderBy(col("Total").desc)

// Récupération du produit le plus vendu (première ligne)
val topProduit = Vente.first().getAs[String]("Product")

// Affichage du produit le plus vendu
println(s"Le produit le plus vendu est : $topProduit")


first at cmd36.sc:6 
 (kill)

0 / 12

first at cmd36.sc:6 
 (kill)

0 / 12

first at cmd36.sc:6 
 (kill)

0 / 200

first at cmd36.sc:6 
 (kill)

0 / 200

Le produit le plus vendu est : AAA Batteries (4-pack)


Vente: Dataset[Row] = [Product: string, Total: double]
topProduit: String = "AAA Batteries (4-pack)"

### Les piles AAA sont des articles couramment utilisés dans de nombreux appareils électroniques en temoigne la reponse a la question 5.

In [43]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.sql.{SparkSession, DataFrame}



val cleanedData: DataFrame = df.na.drop()


// Préparation des données
val assembler = new VectorAssembler()
  .setInputCols(Array("Product", "Quantity Ordered", "Price Each"))
  .setOutputCol("features")

val preparedData: DataFrame = assembler.transform(cleanedData)
  .select("features", "Cable USB-C") // Colonne "Cable USB-C" contient la variable cible (0 pour non, 1 pour oui)

// Division des données en ensembles d'entraînement et de test
val Array(trainingData, testData) = preparedData.randomSplit(Array(0.7, 0.3))

// Création du modèle de régression logistique
val lr = new LogisticRegression()
  .setLabelCol("Cable USB-C")
  .setFeaturesCol("features")

// Entraînement du modèle
val model = lr.fit(trainingData)

// Utilisation du modèle pour faire des prédictions sur les données de test
val predictions = model.transform(testData)

// Calcul de la probabilité moyenne de commande d'un câble USB-C
val probability = predictions.selectExpr("avg(prediction)").first().getDouble(0)

println(s"La probabilité que les prochains clients commandent un câble de chargement USB-C est : $probability")

// Sauvegarde du modèle entraîné pour une utilisation ultérieure
model.save("chemin/vers/votre/modele")

: 